In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
ROOT_PATH = "/content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto"

In [3]:
!pip3 install -q implicit

     |████████████████████████████████| 1.1 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [11]:
import json
with open(f'{ROOT_PATH}/ranking_without_test.json') as json_file:
  ranking_dict = json.load(json_file)

In [12]:
ranking_dict

{'01': [38, 1, 4, 21, 0, 16, 32, 11, 36, 9],
 '02': [9, 1, 0, 11, 16, 21, 4, 32, 38, 36],
 '03': [9, 38, 4, 32, 36, 1, 0, 21, 16, 11],
 '04': [0, 16, 11, 21, 4, 9, 32, 1, 38, 36],
 '05': [9, 32, 1, 38, 36, 4, 11, 16, 0, 21],
 '06': [9, 32, 38, 1, 11, 0, 4, 36, 16, 21],
 '07': [9, 36, 38, 32, 4, 11, 1, 16, 0, 21],
 '08': [0, 9, 32, 1, 4, 16, 11, 38, 36, 21],
 '09': [11, 16, 1, 38, 9, 21, 4, 0, 36, 32],
 '10': [4, 0, 1, 38, 9, 32, 36, 16, 21, 11],
 '11': [9, 4, 11, 0, 16, 1, 21, 32, 36, 38],
 '12': [0, 1, 16, 11, 9, 21, 4, 38, 32, 36],
 '13': [1, 32, 4, 9, 11, 0, 16, 21, 38, 36],
 '14': [4, 38, 9, 16, 11, 1, 36, 0, 32, 21],
 '15': [4, 0, 9, 16, 11, 1, 32, 21, 38, 36],
 '16': [4, 9, 38, 36, 1, 21, 32, 11, 0, 16],
 '17': [4, 38, 36, 21, 11, 32, 9, 1, 16, 0],
 '18': [4, 9, 16, 36, 1, 32, 0, 38, 11, 21],
 '19': [4, 9, 32, 36, 0, 1, 21, 38, 11, 16],
 '20': [4, 11, 16, 32, 9, 36, 0, 21, 38, 1],
 '21': [32, 4, 9, 38, 0, 16, 1, 21, 11, 36],
 '22': [4, 1, 9, 0, 16, 21, 11, 38, 32, 36],
 '23': [1,

In [13]:
files = [str(j).zfill(2) for j in range(1, 33)]

In [14]:
import numpy as np


def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)


def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])
  

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.
    

def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [15]:
def calculate_metrics(recommendations, relevant_animes, k):
    mean_nDCG = 0
    relevant_recommendations = []

    for user_id, anime_list in relevant_animes.items():
        bool_list = np.isin(recommendations[user_id], anime_list)
        int_list = [1 if boolean else 0 for boolean in bool_list]

        ndcg_metric = ndcg_at_k(int_list, k)
        relevant_recommendations.append(int_list)

        mean_nDCG += ndcg_metric

    mean_nDCG /= len(relevant_animes)
    mAP = mean_average_precision(relevant_recommendations)

    return mAP, mean_nDCG

---

In [18]:
LABELS_PATH = f"{ROOT_PATH}/features-and-labels/labels_all.npy"

In [27]:
import numpy as np


with open(LABELS_PATH, 'rb') as file:
    labels = np.load(file)

In [20]:
TEST_SIZE = 10
NUM_VIDEOS = 40

np.random.seed(0)

idxs = list(range(NUM_VIDEOS))

TEST_VIDEO_IDXS = np.random.choice(idxs, TEST_SIZE, replace=False)

In [29]:
test_labels = labels[:, TEST_VIDEO_IDXS]
test_labels.shape

(32, 10)

In [128]:
np.sum(test_labels)

212

In [32]:
relevant_videos = {}
for index,row in enumerate(test_labels):
  idx_relevant_videos = []
  for index_v,item in enumerate(row):
    if item == 1:
      idx_relevant_videos.append(index_v)
  relevant_videos[index] = idx_relevant_videos

---

In [132]:
from scipy.sparse import csr_matrix
import json

MATRIX_PATH = f"{ROOT_PATH}/matrix.json"
with open(MATRIX_PATH, 'r') as file:
    matrix = json.load(file)

user_item_sparse_matrix = csr_matrix([row for row in matrix.values()]).T
user_item_sparse_permuted = csr_matrix(np.random.randint(0, 2, (32, 10))).T

In [24]:
user_item_sparse_matrix

<10x32 sparse matrix of type '<class 'numpy.longlong'>'
	with 231 stored elements in Compressed Sparse Column format>

In [25]:
import implicit 

factors = 25
reg_param = 0.01
iterations = 20
use_gpu = False

als_model = implicit.als.AlternatingLeastSquares(
    factors=factors, 
    regularization=reg_param, 
    use_gpu=use_gpu, 
    iterations=iterations, 
)

In [95]:
reg_param = 0.01
iterations = 20
use_gpu = False

factor_list = [50, 100, 200, 500, 1000]
k_list = [3,5,7]
results_3 = {}  # factors: time, mAP, nCDG
results_5 = {}
results_7 = {}
for factors in factor_list:

    als_model = implicit.als.AlternatingLeastSquares(
        factors=factors, 
        regularization=reg_param, 
        use_gpu=use_gpu, 
        iterations=iterations, 
    )

    als_model.fit(user_item_sparse_matrix)

    als_recommendations = {
      user_id: get_recommendation(als_model, user_id, user_item_sparse_matrix.T) for \
      user_id in range(0,32)
    }
    for k in k_list:
      als_mAP, als_nDCG = calculate_metrics(als_recommendations, relevant_videos, k)
      if k == 3:
        results_3[factors] = [als_mAP, als_nDCG]
      if k == 5:
        results_5[factors] = [als_mAP, als_nDCG]
      else:
        results_7[factors] = [als_mAP, als_nDCG]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [97]:
results_3

{50: [0.4654427083333333, 0.4391157701503428],
 100: [0.43728794642857144, 0.4192568643532025],
 200: [0.4659982638888889, 0.4518606507149763],
 500: [0.4360379464285714, 0.41796895937213663],
 1000: [0.4412462797619048, 0.4345539991839255]}

In [138]:
results_5

{50: [0.4654427083333333, 0.4584240569705015],
 100: [0.43728794642857144, 0.4364633436044214],
 200: [0.4659982638888889, 0.45850584895457913],
 500: [0.4360379464285714, 0.4367448591342713],
 1000: [0.4412462797619048, 0.4396020362494564]}

In [139]:
results_7

{50: [0.4654427083333333, 0.4869485898148123],
 100: [0.43728794642857144, 0.4652303108531579],
 200: [0.4659982638888889, 0.48717996000727093],
 500: [0.4360379464285714, 0.4655118263830078],
 1000: [0.4412462797619048, 0.4683690034981929]}

In [67]:
def get_recommendation(model, user, user_items, anime_inverse_map=None):
    if anime_inverse_map is None:
        return [i for i, _ in model.recommend(user, user_items)]
    return [anime_inverse_map[i] for i, _ in model.recommend(user, user_items)]

In [70]:
als_recommendations = {
      user_id: get_recommendation(als_model, user_id, user_item_sparse_matrix.T) for \
      user_id in range(0,32)
    }

In [71]:
als_recommendations

{0: [0],
 1: [0],
 2: [4, 0, 9, 1, 2],
 3: [],
 4: [0],
 5: [0],
 6: [],
 7: [9, 0],
 8: [],
 9: [2, 0, 1, 9],
 10: [6, 2, 9, 1, 0],
 11: [1, 9, 0],
 12: [5, 1, 9, 0, 2, 6, 4, 7, 3],
 13: [7, 2, 9, 1, 0, 4],
 14: [0],
 15: [],
 16: [5, 1, 9, 0, 2, 6, 4, 7, 3],
 17: [1, 0],
 18: [0],
 19: [0],
 20: [0],
 21: [6, 7, 9, 1, 4, 0, 2],
 22: [3, 9, 1, 0, 6, 2, 4, 7],
 23: [0],
 24: [],
 25: [],
 26: [1, 9, 0],
 27: [2, 0, 1, 9],
 28: [],
 29: [1, 9, 0],
 30: [0],
 31: [5, 1, 9, 0, 2, 6, 4, 7, 3]}

In [35]:
als_scores = np.matmul(als_model.user_factors, als_model.item_factors.T)
als_scores

array([[ 7.8111631e-04,  9.9994230e-01,  1.0002440e+00,  9.9984330e-01,
         9.9995345e-01,  9.9971342e-01,  9.9969172e-01,  9.9983716e-01,
         1.0001477e+00,  1.0000836e+00],
       [ 7.8110932e-04,  9.9994236e-01,  1.0002443e+00,  9.9984342e-01,
         9.9995345e-01,  9.9971348e-01,  9.9969155e-01,  9.9983710e-01,
         1.0001478e+00,  1.0000834e+00],
       [ 6.9950911e-05, -7.1431808e-05, -1.8665736e-04,  1.0002775e+00,
         6.3279825e-03,  1.0002228e+00,  1.0005759e+00,  9.9336463e-01,
         1.0000846e+00, -6.6497552e-05],
       [ 9.9893922e-01,  9.9964666e-01,  9.9971718e-01,  1.0002694e+00,
         1.0000641e+00,  1.0004449e+00,  1.0003227e+00,  1.0001849e+00,
         1.0000206e+00,  9.9953645e-01],
       [ 7.8106415e-04,  9.9994242e-01,  1.0002441e+00,  9.9984342e-01,
         9.9995357e-01,  9.9971342e-01,  9.9969161e-01,  9.9983716e-01,
         1.0001479e+00,  1.0000834e+00],
       [ 7.8101177e-04,  9.9994230e-01,  1.0002441e+00,  9.9984336e-01,
   

In [36]:
ALS_THRESHOLD = 0.9

als_recs = (als_scores > ALS_THRESHOLD).astype(int)
als_recs

array([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1,

In [89]:
calculate_metrics(als_recommendations, relevant_videos, )

(0.427828621031746, 0.4780690126751739)

In [62]:
relevant_videos

{0: [0, 1, 2, 5, 6, 7, 9],
 1: [0, 2, 4, 5, 6, 7, 8, 9],
 2: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 3: [0, 2, 3, 4, 6, 7, 8, 9],
 4: [2, 4, 5, 7, 8, 9],
 5: [1, 3, 5, 6, 8, 9],
 6: [0, 1, 2, 3, 4, 5, 6, 7, 8],
 7: [1, 2, 4, 6, 7, 9],
 8: [0, 4, 5, 7, 9],
 9: [1, 2, 3, 4, 7, 8],
 10: [0, 1, 2, 3, 4, 8],
 11: [2, 4, 5, 6, 7, 8, 9],
 12: [0, 2, 3, 4, 6, 7, 8],
 13: [0, 2, 3, 4, 6, 7, 9],
 14: [2, 3, 4, 5, 6, 7, 8, 9],
 15: [2, 3, 7, 9],
 16: [3, 4, 5, 6, 8, 9],
 17: [3, 4, 8],
 18: [1, 3, 4, 5, 7, 8, 9],
 19: [0, 3, 4, 6, 7, 8, 9],
 20: [0, 3, 4, 5, 6, 7, 8, 9],
 21: [0, 2, 3, 4, 5, 6, 8, 9],
 22: [2, 3, 4, 7, 8, 9],
 23: [1, 2, 3, 4, 5, 9],
 24: [0, 1, 4, 5, 6, 7, 8, 9],
 25: [1, 4, 6, 7, 8, 9],
 26: [0, 1, 4, 5, 6, 9],
 27: [3, 4, 5, 7, 8, 9],
 28: [3, 4, 8, 9],
 29: [3, 4, 5, 6, 8, 9],
 30: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 31: [0, 3, 4, 5, 7, 8, 9]}

---

In [111]:
reg_param = 0.01
iterations = 20
use_gpu = False

k_list = [3,5,7]
factor_list = [50, 100, 200, 500, 1000]

results_bpr_3 = {}  # factors: time, mAP, nCDG
results_bpr_5 = {}
results_bpr_7 = {}
for factors in factor_list:
    bpr_model = implicit.bpr.BayesianPersonalizedRanking(
        factors=factors, 
        regularization=reg_param, 
        use_gpu=use_gpu, 
        iterations=iterations, 
    )

    bpr_model.fit(user_item_sparse_matrix)

    bpr_recommendations = {
      user_id: get_recommendation(bpr_model, user_id, user_item_sparse_matrix.T) for \
      user_id in range(0,32)
    }

    for k in k_list:
      bpr_mAP, bpr_nDCG = calculate_metrics(bpr_recommendations, relevant_videos, k)
      if k == 3:
        results_bpr_3[factors] = [bpr_mAP, bpr_nDCG]
      if k == 5:
        results_bpr_5[factors] = [bpr_mAP, bpr_nDCG]
      else:
        results_bpr_7[factors] = [bpr_mAP, bpr_nDCG]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [115]:
results_bpr_3

{50: [0.49683283730158734, 0.48979798870714436],
 100: [0.49553075396825397, 0.4859578294599889],
 200: [0.4316604662698412, 0.42980671404626347],
 500: [0.4942286706349207, 0.4824655086130388],
 1000: [0.4940178571428572, 0.4763774045730286]}

---

In [136]:
results_bpr_5

{50: [0.49683283730158734, 0.4991875804584995],
 100: [0.49553075396825397, 0.5005123539996527],
 200: [0.4316604662698412, 0.4480157694081478],
 500: [0.4942286706349207, 0.4979883609233599],
 1000: [0.4940178571428572, 0.5027495941764929]}

In [137]:
results_bpr_7

{50: [0.49683283730158734, 0.5151722055118018],
 100: [0.49553075396825397, 0.5166262419127369],
 200: [0.4316604662698412, 0.4573654327937568],
 500: [0.4942286706349207, 0.5140230515878483],
 1000: [0.4940178571428572, 0.5153305632846437]}

In [119]:
def random_recommendation(num_users, num_items, k):
    np.random.seed(0)
    if k > num_items:
      raise ValueError

    item_idxs = list(range(num_items))

    recs = {}
    for user in range(num_users):
        np.random.shuffle(item_idxs)
        recs[user] = [c for c in item_idxs[:k]]

    return recs

In [124]:
r_rec_3 = random_recommendation(32,10,5)

In [126]:
r_rec_3

{0: [2, 8, 4, 9, 1],
 1: [9, 6, 8, 4, 5],
 2: [8, 4, 3, 5, 0],
 3: [9, 4, 7, 3, 2],
 4: [6, 7, 8, 2, 4],
 5: [7, 0, 3, 9, 8],
 6: [9, 3, 0, 5, 4],
 7: [9, 7, 0, 5, 2],
 8: [3, 6, 4, 7, 2],
 9: [6, 4, 1, 3, 9],
 10: [2, 0, 5, 4, 6],
 11: [6, 9, 7, 5, 0],
 12: [0, 8, 9, 2, 4],
 13: [3, 6, 4, 8, 1],
 14: [3, 7, 2, 6, 1],
 15: [4, 0, 5, 9, 3],
 16: [0, 3, 1, 6, 5],
 17: [5, 0, 3, 2, 7],
 18: [7, 0, 9, 1, 6],
 19: [4, 5, 0, 9, 8],
 20: [0, 8, 9, 1, 5],
 21: [9, 7, 6, 5, 0],
 22: [6, 0, 7, 5, 3],
 23: [2, 6, 7, 3, 0],
 24: [9, 6, 0, 2, 5],
 25: [6, 0, 5, 1, 4],
 26: [3, 7, 0, 1, 5],
 27: [3, 0, 7, 8, 9],
 28: [9, 1, 0, 3, 2],
 29: [5, 9, 3, 0, 2],
 30: [4, 6, 8, 3, 0],
 31: [8, 3, 5, 0, 6]}

In [134]:
bpr_model = implicit.bpr.BayesianPersonalizedRanking(
      factors=1000, 
      regularization=reg_param, 
      use_gpu=use_gpu, 
      iterations=iterations, 
    )

bpr_model.fit(user_item_sparse_permuted)

bpr_recommendations = {
    user_id: get_recommendation(bpr_model, user_id, user_item_sparse_matrix.T) for \
    user_id in range(0,32)
    }

  0%|          | 0/20 [00:00<?, ?it/s]

In [156]:
als_model = implicit.als.AlternatingLeastSquares(
        factors=10, 
        regularization=reg_param, 
        use_gpu=use_gpu, 
        iterations=iterations, 
    )

als_model.fit(user_item_sparse_permuted)

als_recommendations = {
    user_id: get_recommendation(als_model, user_id, user_item_sparse_matrix.T) for \
    user_id in range(0,32)
    }

  0%|          | 0/20 [00:00<?, ?it/s]

random

In [141]:
calculate_metrics(bpr_recommendations, relevant_videos,3) 

(0.39034970238095235, 0.3482129488953555)

In [142]:
calculate_metrics(bpr_recommendations, relevant_videos,5)

(0.39034970238095235, 0.3946273928183488)

In [155]:
calculate_metrics(bpr_recommendations, relevant_videos,7)

(0.39034970238095235, 0.4232447818587043)

In [157]:
calculate_metrics(als_recommendations, relevant_videos,3)

(0.428250248015873, 0.41322167423447465)

In [158]:
calculate_metrics(als_recommendations, relevant_videos,5)

(0.428250248015873, 0.42656427388057233)

In [159]:
calculate_metrics(als_recommendations, relevant_videos,7)

(0.428250248015873, 0.457276650879358)